In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install transformers pandas scikit-learn torch

In [3]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
# 1. Preparação dos Dados

CLASSES_SENTIMENTO = ['positivo', 'neutro', 'negativo', 'boa']
CLASSES_FILTRO = ['onca', 'caseiro', 'fake news']

MODEL_NAME = 'neuralmind/bert-base-portuguese-cased'
NUM_CLASSES = 3

# mapeando
label_map = {'negativo': 0, 'neutro': 1, 'positivo': 2}
id_to_label = {0: 'negativo', 1: 'neutro', 2: 'positivo'}
CLASSES_FOCADAS = list(label_map.keys())

try:
  df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/data.csv', sep=',')
except FileNotFoundError:
  print("ERRO: o arquivo 'data.csv' não foi encontrado.")
  raise

df.columns = df.columns.str.strip()

def get_sentiment_label(row):
  for col in CLASSES_FILTRO:
    sentiment_string = str(row[col]).lower().strip()
    if sentiment_string in CLASSES_SENTIMENTO:
      return sentiment_string

  outras_cols = ['notícia', 'ironia', 'Conservacionista']
  for col in outras_cols:
    sentiment_string = str(row.get(col, '')).lower().strip()
    if sentiment_string in CLASSES_SENTIMENTO:
      return sentiment_string

  return pd.NA

df['sentiment'] = df.apply(get_sentiment_label, axis=1)

# Ação Corretiva: Mudar 'boa' para 'positivo' nos dados ANTES do mapeamento
df['sentiment'] = df['sentiment'].replace({'boa': 'positivo'})

# Limpeza
df.dropna(subset=['sentiment', 'comment_text'], inplace=True)
df = df.copy()

df.drop_duplicates(subset=['comment_text'], inplace=True)
df = df[df['comment_text'].str.strip() != '']

# conversão para ID
df['label_id'] = df['sentiment'].map(label_map)
df = df[df['label_id'].notna()]
df.dropna(subset=['label_id'], inplace=True)
df['label_id'] = df['label_id'].astype(int)

# divisão
if len(df) < 6:
  print(f"ERRO: A filtragem resultou em apenas {len(df)} amostras.")
  raise ValueError("Dados insuficientes para classificação")

# (70% treino, 15% validação, 15% teste)
try:
  df_train_val, df_test = train_test_split(df, test_size=0.15, random_state=42, stratify=df['label_id'])
  df_train, df_val = train_test_split(df_train_val, test_size=(0.15 / 0.85), random_state=42, stratify=df_train_val['label_id'])
except ValueError as e:

    print("\nAVISO: Não foi possível aplicar 'stratify'.")
    df_train_val, df_test = train_test_split(df, test_size=0.15, random_state=42)
    df_train, df_val = train_test_split(df_train_val, test_size=(0.15 / 0.85), random_state=42)

print(f"Número de classes de Sentimentos: {NUM_CLASSES} ({CLASSES_FOCADAS})")
print(f"Total de amostras: {len(df)}")
print(f"Treino: {len(df_train)} amostras | Validação: {len(df_val)} amostras | Teste: {len(df_test)} amostras")


In [ ]:
# 2. Tokenização

MAX_LEN = 128
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

class NewsDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
    return item

  def __len__(self):
    return len(self.labels)

def tokenize_data(data):
  return tokenizer.batch_encode_plus(
      data['comment_text'].tolist(),
      max_length=MAX_LEN,
      padding='max_length',
      truncation=True,
      return_tensors='pt'
  )
# Tokenizar e criar Datasets
train_encodings = tokenize_data(df_train)
val_encodings = tokenize_data(df_val)
test_encodings = tokenize_data(df_test)


train_dataset = NewsDataset(train_encodings, df_train['label_id'].tolist())
val_dataset = NewsDataset(val_encodings, df_val['label_id'].tolist())
test_dataset = NewsDataset(test_encodings, df_test['label_id'].tolist())

print("Tokenização concluída e Datasets criados com sucesso.")
print(f"Estrutura do Dataset de Treino: {len(train_dataset)} amostras")
print(f"Estrutura do Dataset de Validação: {len(val_dataset)} amostras")

In [ ]:
# 3. Modelo / 4.Treinamento

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

# carregar o modelo BERT
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_CLASSES
    ).to(device)

def compute_metrics(p):
  preds = np.argmax(p.predictions, axis=1)
  return {'accuracy': accuracy_score(p.label_ids, preds)}

# configuração de teinamento
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='.logs',
    logging_steps=50,
    learning_rate=2e-5,
    load_best_model_at_end=False,
    report_to="none",
    seed=42
)

# iniciar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

print("\nIniciando o treinamento (pode levar vários minutos)...")

all_eval_metrics = []
all_train_loss = []
total_epochs = 10

for epoch in range(1, total_epochs + 1):
  print(f"\n--- Época {epoch}/{total_epochs} ---")

  trainer.train()

  # coleta o Loss de treino
  train_logs = pd.DataFrame(trainer.state.log_history)
  # filtra e pega o último log de 'loss' de treino
  last_train_log = train_logs[train_logs['loss'].notna()].iloc[-1]
  last_train_loss = last_train_log['loss']
  all_train_loss.append(last_train_loss)

  eval_metrics = trainer.evaluate()
  all_eval_metrics.append(eval_metrics)

  eval_loss = eval_metrics['eval_loss']
  eval_accuracy = eval_metrics['eval_accuracy']

  print(f"Loss Treino (Último Step): {last_train_loss:.4f}")
  print(f"Loss Validação: {eval_loss:.4f} | Acurácia Validação: {eval_accuracy:.4f}")

# mostrar logs de resultadso
print("\n--- Resultados do Treinamento ---")
results_df = pd.DataFrame({
    'Época': range(1, total_epochs + 1),
    'Loss Treino': all_train_loss,
    'Loss Validação': [m['eval_loss'] for m in all_eval_metrics],
    'Acurácia Validação': [m['eval_accuracy'] for m in all_eval_metrics]
})
print(results_df)

# plotar o gráfico de evolução do Loss
plt.figure(figsize=(10, 6))
plt.plot(results_df['Época'], results_df['Loss Treino'], label='Treino Loss', linestyle='--')
plt.plot(results_df['Época'], results_df['Loss Validação'], label='Validação Loss', marker='o')
plt.title('Evolução do Loss (Treino vs. Validação)')
plt.xlabel('Época')
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# 5. Avaliação

# avaliação no conjunto de teste
print("\n--- Iniciando Predições no Conjunto de Teste ---")

predictions = trainer.predict(test_dataset)
test_preds = np.argmax(predictions.predictions, axis=1)
test_true = predictions.label_ids

class_ids = list(id_to_label.keys())

# mostrar o Classification Report
print("\n--- Relatório de Classificação no Conjunto de Teste ---")
print(classification_report(
    test_true,
    test_preds,
    target_names=list(id_to_label.values()),
    digits=4,
    zero_division=0
))

print(f"Acurácia Geral no Teste: {accuracy_score(test_true, test_preds):.4f}\n")

# mostrar exemplos de Erros
df_test_final = df_test.copy()
df_test_final['predicted_id'] = test_preds
df_test_final['predicted_label'] = df_test_final['predicted_id'].map(id_to_label)

errors = df_test_final[df_test_copy['label_id'] != df_test_final['predicted_id']]
print(f"\n--- {len(errors)} Exemplos de Erros de Classificação ---")


print("Cinco Erros Aleatórios:")
if len(errors) > 0:
  for index, row in errors.sample(min(5, len(errors))).iterrows():
    print(f"Texto: **{row['comment_text'][:80]}...**")
    print(f" Rótulo Real: {row['sentiment']}")
    print(f" Predição: {row['predicted_label']}\n")
else:
  print("Nenhum erro encontrado.")
